In [2]:
import os

In [3]:
%pwd # current dir

'c:\\Users\\15600\\Desktop\\PY\\kidney-disease-classification-project\\research'

In [4]:
os.chdir("../") # to the prev. dir

In [5]:
%pwd

'c:\\Users\\15600\\Desktop\\PY\\kidney-disease-classification-project'

## Entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) # simplify the class defination （no need to write def __init__(self)）
class DataIngestionConfig:
    root_dir: Path
    src_URL: str
    local_data_file: Path
    unzip_dir: Path

## Configuration

In [7]:
from src.KDClassifier.constants import *
from src.KDClassifier.utils.common import read_yaml, create_directories


In [8]:
class ConfigurationManager:
    '''
    read the config.yaml
    return data_ingestion_config
    '''
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                       params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            src_URL=config.src_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [9]:
data_config = ConfigurationManager()


[2024-02-08 10:13:26,350: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-08 10:13:26,352: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-08 10:13:26,353: INFO: common: created directory at: artifacts]


In [10]:
srcURL = data_config.get_data_ingestion_config()


[2024-02-08 10:13:26,363: INFO: common: created directory at: artifacts/data_ingestion]


In [11]:
srcURL.src_URL

'https://drive.google.com/file/d/1jGFKjl11IOSnhUx-HJnOsIPwxJWl3q-G/view?usp=drive_link'

## Components

In [12]:
import os
import zipfile
import gdown
from src.KDClassifier import logger
from src.KDClassifier.utils.common import get_size


In [13]:
class DataIngestion:
    '''
    download and extract data
    '''
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self) -> str:
      try:
          dataset_url = self.config.src_URL
          zip_download_dir = self.config.local_data_file
          os.makedirs("artifacts/data_ingestion", exist_ok=True)
          logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

          file_id = dataset_url.split('/')[-2]
          prefix = "https://drive.google.com/uc?/export=download&id="
          gdown.download(prefix+file_id, zip_download_dir)
          logger.info("Download finished!")
      except Exception as e:
          raise e
    
    def extract_zip_file(self):
        '''
        zip_file_path: str
        extracts zip file into data dir
        return None  
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        logger.info("extracting data")
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info("extract finished!")

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-02-08 10:21:16,314: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-08 10:21:16,315: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-08 10:21:16,316: INFO: common: created directory at: artifacts]
[2024-02-08 10:21:16,317: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-08 10:21:16,318: INFO: 2657407201: Downloading data from https://drive.google.com/file/d/1jGFKjl11IOSnhUx-HJnOsIPwxJWl3q-G/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1jGFKjl11IOSnhUx-HJnOsIPwxJWl3q-G
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1jGFKjl11IOSnhUx-HJnOsIPwxJWl3q-G&confirm=t&uuid=ef657715-57e0-47db-81ee-03626cd26298
To: c:\Users\15600\Desktop\PY\kidney-disease-classification-project\artifacts\data_ingestion\data.zip
100%|██████████| 941M/941M [01:16<00:00, 12.3MB/s] 

[2024-02-08 10:22:33,896: INFO: 2657407201: Download finished!]
